In [1]:
from utils import *

import folium
import geopandas as gpd

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_log_error, mean_absolute_error
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

from xgboost import XGBRegressor

## Data description

In [2]:
macro_df = pd.read_csv('data/macro.csv', parse_dates=['timestamp'])
train_df = pd.read_csv('data/train.csv', index_col='id', parse_dates=['timestamp'])
test_df = pd.read_csv('data/test.csv', index_col='id', parse_dates=['timestamp'])
tverskoe_issue_fix(train_df)
tverskoe_issue_fix(test_df)

Fix:  550
Fix:  149


## 1. Data preprocessing
## I part (encoding and correcting mistakes)

### Macro dataset

In [3]:
macro_df['child_on_acc_pre_school'] = macro_df['child_on_acc_pre_school'].str.replace('#!', 'nan')
for column in macro_df.select_dtypes('object').columns:
    macro_df[column] = macro_df[column].str.replace(',', '.')
    macro_df[column] = macro_df[column].astype(float)

if not len(macro_df.select_dtypes('object').columns):
    print('OK')

OK


### Train dataset

In [4]:
train_df = encode(train_df)

### Test dataset

In [5]:
test_df = encode(test_df)

## II part (Filling missing values)

XGBRegressor model handles `np.NaN` values itself

## 2. Encoding `sub_area` feature

In [6]:
train_df['is_train'] = 1
test_df['is_train'] = 0

coords_df = pd.read_csv('data/coords.csv', index_col='id')
all_df = pd.concat([train_df, test_df])

all_df['latitude'] = coords_df['latitude']
all_df['longitude'] = coords_df['longitude']

## 3. Removing outliers

In [7]:
all_df = remove_outliers(all_df)

In [8]:
train_df = all_df[all_df['is_train'] == 1].drop(['is_train'], axis=1)
test_df = all_df[all_df['is_train'] == 0].drop(['is_train', 'price_doc'], axis=1)

## 4. Modeling

### `product_type == 'OwnerOccupier'`

In [13]:
train_df_1 = train_df[train_df['product_type'] == 1]
test_df_1 = test_df[test_df['product_type'] == 1]

X_1 = train_df_1.drop(['sub_area', 'price_doc'], axis=1).copy()
y_1 = np.log1p(train_df_1['price_doc'])
X_test_1 = test_df_1.drop(['sub_area'], axis=1).copy()

In [14]:
X_train_1, X_val_1, y_train_1, y_val_1 = train_test_split(X_1, y_1, test_size=0.2, random_state=42)
X_train_1.shape, X_val_1.shape

((8795, 293), (2199, 293))

In [33]:
params = {'n_estimators': 10000,
          'base_score': 7,
          'booster': 'gbtree',
          'max_depth': 6,
          'eval_metric': 'rmse',
          'learning_rate': 0.05,
          # 'reg_alpha': 30,
          # 'min_child_weight': 10,
          'subsample': 0.8,
          'colsample_bytree': 0.8,
          'seed': 42,
          'nthread': -1
          }

model_1 = XGBRegressor(objective='reg:squarederror', **params)

model_1.fit(X_train_1, y_train_1, eval_set=[(X_train_1, y_train_1), (X_val_1, y_val_1)], early_stopping_rounds=50)


[0]	validation_0-rmse:8.14571	validation_1-rmse:8.13362
[1]	validation_0-rmse:7.73913	validation_1-rmse:7.72732
[2]	validation_0-rmse:7.35283	validation_1-rmse:7.34135
[3]	validation_0-rmse:6.98585	validation_1-rmse:6.97455
[4]	validation_0-rmse:6.63721	validation_1-rmse:6.62629
[5]	validation_0-rmse:6.30608	validation_1-rmse:6.29559
[6]	validation_0-rmse:5.99149	validation_1-rmse:5.98099
[7]	validation_0-rmse:5.69257	validation_1-rmse:5.68255
[8]	validation_0-rmse:5.40863	validation_1-rmse:5.39908
[9]	validation_0-rmse:5.13885	validation_1-rmse:5.12948
[10]	validation_0-rmse:4.88268	validation_1-rmse:4.87361
[11]	validation_0-rmse:4.63919	validation_1-rmse:4.63035
[12]	validation_0-rmse:4.40787	validation_1-rmse:4.39926
[13]	validation_0-rmse:4.18816	validation_1-rmse:4.17976
[14]	validation_0-rmse:3.97952	validation_1-rmse:3.97141
[15]	validation_0-rmse:3.78129	validation_1-rmse:3.77329
[16]	validation_0-rmse:3.59297	validation_1-rmse:3.58514
[17]	validation_0-rmse:3.41407	validation

XGBRegressor(base_score=7, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, eval_metric='rmse',
             gamma=0, gpu_id=-1, importance_type='gain',
             interaction_constraints='', learning_rate=0.05, max_delta_step=0,
             max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=10000, n_jobs=8,
             nthread=-1, num_parallel_tree=1, random_state=42, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, seed=42, subsample=0.8,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [34]:
y_pred_train = np.abs(model_1.predict(X_train_1))
y_pred_val = np.abs(model_1.predict(X_val_1))
print('___________________________________')
print('      RMSLE\n')
print('      TRAIN_1:', np.sqrt(mean_squared_log_error(np.expm1(y_train_1), np.expm1(y_pred_train))))
print(' VALIDATION_1:', np.sqrt(mean_squared_log_error(np.expm1(y_val_1), np.expm1(y_pred_val))))
print('___________________________________')
print('      MAE\n')
print('      TRAIN_1:', mean_absolute_error(np.expm1(y_train_1), np.expm1(y_pred_train)))
print(' VALIDATION_1:', mean_absolute_error(np.expm1(y_val_1), np.expm1(y_pred_val)))

___________________________________
      RMSLE

      TRAIN_1: 0.03522042341278136
 VALIDATION_1: 0.0915284365683138
___________________________________
      MAE

      TRAIN_1: 160442.81621660042
 VALIDATION_1: 384396.1815029559


### `product_type == 'Investment'`

In [17]:
create_submission(model, X_test)

In [30]:
submission = pd.read_csv('data/sample_submission.csv')
pred = model.predict(X_test)
if len(pred[pred < 0]):
    print('WARNING: NEGATIVE PREDICTIONS')
    pred = np.abs(pred)
submission['price_doc'] = pred * 0.85
submission.to_csv('submission.csv', index=False)


In [31]:
!kaggle competitions submit -c sberbank-russian-housing-market -f submission.csv -m "XGBRegressor w/out outliers"

100%|████████████████████████████████████████| 121k/121k [00:01<00:00, 63.2kB/s]
Successfully submitted to Sberbank Russian Housing Market

In [29]:
# !kaggle competitions submissions -c sberbank-russian-housing-market